In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Descargamos el archivo
!git clone https://github.com/jwackito/TallerLLMs

Cloning into 'TallerLLMs'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 6), reused 14 (delta 1), pack-reused 0
Receiving objects: 100% (26/26), 35.67 MiB | 15.46 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (10/10), done.


In [4]:
#Abrimos el archivo con los nombres
words = open('/content/TallerLLMs/02/data/names.txt', 'r').read().splitlines()
words[:8]

['Aaren', 'Aarika', 'Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie']

In [5]:
#¿Cuantos nombres tenemos?
len(words)

4945